In [2]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.24s/it]
Device set to use cuda:0


In [1]:
import os
os.chdir('../')


In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.chdir(os.path.join(os.getcwd(), '../'))

In [3]:
df = pd.read_parquet('protclip_embed_dataset.parquet')

In [4]:
df.head()

,organism,organism_id,name,evidence,function,id,textual_embedding,sequence,proteinclip_embed
0,Homo sapiens (Human),9606,EPHA7,1,Receptor tyrosine kinase which binds promiscuo...,Q15375,"[-0.015253728, 0.016237658, -0.016555615, 0.02...",MVFQTRYPSWIILCYIWLLRFAHTGEAQAAKEVLLLDSKAQQTELE...,"[0.16681893169879913, 0.105362169444561, 0.011..."
1,Homo sapiens (Human),9606,ANXA8,1,This protein is an anticoagulant protein that ...,P13928,"[-0.008352073, 0.00474287, 0.006541474, -0.002...",MAWWKSWIEQEGVTVKSSSHFNPDPDAETLYKAMKGIGTNEQAIID...,"[-0.10789338499307632, 0.06770769506692886, -0..."
2,Homo sapiens (Human),9606,DPY19L2P1,2,Probable C-mannosyltransferase that mediates C...,Q6NXN4,"[-0.00039709447, -0.02393247, -0.014100584, 0....",MKKQGVNPKPLQSSRPSPSKRPYGASPARELEVEKSALGGGKLPGG...,"[0.026129087433218956, 0.1801730990409851, 0.0..."
3,Homo sapiens (Human),9606,NR1D1,1,Transcriptional repressor which coordinates ci...,P20393,"[0.008172105, -0.0116752, -0.016805198, -0.005...",MTTLDSNNNTGGVITYIGSSGSSPSRTSPESLYSDNSNGSFQSLTQ...,"[-0.016533901914954185, -0.0384892001748085, 0..."
4,Homo sapiens (Human),9606,SLC15A2,1,Proton-coupled amino-acid transporter that tra...,Q16348,"[0.0031013805, -0.0019497981, -0.003831747, 0....",MNPFQKNESKETLFSPVSIEEVPPRPPSPPKKPSPTICGSNYPLSI...,"[0.08691893517971039, 0.07942728698253632, -0...."


In [3]:
args = {
    "sequence_embeddings": "../data/esm2/sequence_embedding_t33_0_2000.pkl",
    'structural_embed':'../data/structural/structural_embeddings_0_7500.pkl',
    "functional_embeddings_file": "protclip_embed_dataset.parquet",
    'unitnorm': True,
    'batch_size': 256,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [4]:
pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [5]:
structural_embeddings = None
with open(args['structural_embed'], "rb") as f:
    structural_embeddings = pickle.load(f)
structural_embeddings = dict(zip(
    structural_embeddings["protein_id"],
    structural_embeddings["embedding"]
))
list(structural_embeddings.items())[0]

('Q15375',
 array([ -5.27637  ,   9.696514 ,  11.407264 , ...,  -5.5494313,
        -11.50502  , -10.721539 ], dtype=float32))

In [9]:
functional_embeddings = None
df = pd.read_parquet('text+ESM+protclip.parquet')
functional_embeddings: dict[str, str] = dict(df[["id", "textual_embedding"]].values)
sequence_embeddings : dict[str, str] = dict(df[["id", "ESM_embed"]].values)

In [8]:
df.columns

Index(['organism', 'organism_id', 'name', 'evidence', 'function', 'id',
       'textual_embedding', 'sequence', 'proteinclip_embed', 'ESM_embed'],
      dtype='object')

In [13]:
sequence_embeddings = None
with open(args['sequence_embeddings'], "rb") as f:
    sequence_embeddings = pickle.load(f)

list(sequence_embeddings.items())[0]

('Q15375',
 array([ 0.00049137, -0.04332284, -0.01638226, ..., -0.10839974,
         0.02102051,  0.13916047], dtype=float32))

In [14]:
functional_embeddings = None
df = pd.read_parquet(args["functional_embeddings_file"])
functional_embeddings: dict[str, str] = dict(df[["id", "textual_embedding"]].values)


In [10]:
shared_keys = sorted(set(sequence_embeddings.keys()).intersection(set(structural_embeddings.keys())).intersection(functional_embeddings.keys()))


In [11]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [12]:
from proteinclip.data_utils import CLIPDataset3D
dset = data_utils.CLIPDataset3D(
    triples=shared_keys,
    map1=functional_embeddings,
    map2=sequence_embeddings,
    map3=structural_embeddings,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings: 100%|██████████| 7493/7493 [00:00<00:00, 17867.70it/s]
INFO:root:Trimmed 0 triples with missing/zero embeddings; 7493 remain.


In [13]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [14]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [15]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
input_dim_3 = next(iter(train_dl))["x_3"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding3D
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    input_dim_3=input_dim_3,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

In [16]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [17]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA H200') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_

/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (27) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 27/27 [00:00<00:00, 51.08it/s, v_num=2509883, train_loss=3.540, val_loss=4.410]

Metric val_loss improved. New best score: 4.412


Epoch 1: 100%|██████████| 27/27 [00:00<00:00, 41.83it/s, v_num=2509883, train_loss=3.290, val_loss=4.110]

...

Epoch 524: 100%|██████████| 27/27 [00:00<00:00, 42.04it/s, v_num=2509883, train_loss=0.305, val_loss=2.180]

Metric val_loss improved by 0.008 >= min_delta = 0.0. New best score: 2.176


Epoch 574: 100%|██████████| 27/27 [00:00<00:00, 42.75it/s, v_num=2509883, train_loss=0.145, val_loss=2.190]

Monitored metric val_loss did not improve in the last 50 records. Best score: 2.176. Signaling Trainer to stop.


Epoch 574: 100%|██████████| 27/27 [00:00<00:00, 42.65it/s, v_num=2509883, train_loss=0.145, val_loss=2.190]


In [18]:



# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "project3d_1_7500.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project3d_2_7500.onnx",
    input_shape=(input_dim_2,),
)
contrastive.model_to_onnx(
    net.project_3,
    "project3d_3_7500.onnx",
    input_shape=(input_dim_3,),
)


2025-04-24 18:58:59.392291819 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3770828, index: 0, mask: {24, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:58:59.392293822 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3770829, index: 1, mask: {48, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:58:59.392300751 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3770830, index: 2, mask: {8, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:58:59.397298959 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3770882, index: 52, mask: {27, }, error code: 22 error msg: Invalid argument. Specify t